In [ ]:
import pyodbc
import paramiko
import xlsxwriter
import win32com.client
import datetime
import getpass

UID=input("Enter Userid :")
PWD=getpass.getpass("Enter password :")
today = datetime.date.today()
first = today.replace(day=1)
cur_mmyy=first.strftime("%b_%Y")
mmyy=first.strftime("%Y_%m")

# connecting to OLDW server
ssh_client=paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh_client.connect(hostname='10.11.45.9',username='{}'.format(UID),password='{}'.format(PWD))

# Please check the sqls in /OLABS_QA/prod/NHI/mnthly/Metrics_Files/DQ_Input_params.txt
stdin,stdout,stderr=ssh_client.exec_command("cat /OLABS_QA/prod/NHI/mnthly/Metrics_Files/DQ_Input_params.txt")
out=stdout.readlines()
for file in out:
    print(file)

Enter Userid :pyanama1


In [ ]:
# executing sqls from parameter files
# These sqls take time and jupiter will go to sleep while sqls are running in unix server.
# Read the log file - /OLABS_QA/prod/NHI/mnthly/output/Run_Sqls_For_Current_vs_Previous.log to see the completed processes

stdin,stdout,stderr=ssh_client.exec_command("nohup sh /OLABS_QA/scripts/Run_Sqls_For_Current_vs_Previous.sh")
out=stdout.readlines()


In [ ]:
# once the above sqls are successful, run this cell to generate excel reports

stdin,stdout,stderr=ssh_client.exec_command("cat /OLABS_QA/prod/NHI/mnthly/output/Run_Sqls_For_Current_vs_Previous.log")
out=stdout.readlines()
for i in out:
    print(i)

#file_names consists of log file pattern and this pattern is used to fetch the log file namev for current month used for respective sheets
file_names={"OLDW_COUNT_TREND":["SQL_NHI_count_trend_mdfid_direct_",""],"ADD_DERIVED_DATA_SOURCE":["SQL_NHI_current_vs_previous_distribution_mdfid_",""],"MC_MCMPARTD":["SQL_MC_MCPD_",""],"DISTR_BASED_ON_DATES":["SQL_OLDW_LIVE_VS_NONLIVE_DISTRIBUTION_",""]}

print("Please verify the below files which are being processed for metrics documents")
for key,value in file_names.items():
    pattern=value[0]
    stdin,stdout,stderr=ssh_client.exec_command("ls -t /OLABS_QA/prod/NHI/mnthly/output/{}{}*.log|head -1".format(pattern,mmyy))
    out=stdout.readlines()
    file=out[0].strip()
    value[1]=file
    print(file)

filepath="C:\\Users\\"+UID+"\\Downloads\\Current_Vs_Previous_Month_Distribution_"+cur_mmyy+".xlsm"
filename="Current_Vs_Previous_Month_Distribution_"+cur_mmyy+".xlsm"
oldwpath="/OLABS_QA/prod/NHI/mnthly/Metrics_Files/Current_vs_Previous/"

#converting numbers and decimals in logs from strings to their respective data types
def clean_num(line):
    for j in range(len(line)):
        line[j]=line[j].strip()
        if(len(line[j])!=0):
            if(line[j][0]=="-"):
                if(line[j][1:].isnumeric()):
                    line[j]=0-int(line[j][1:])
                elif(line[j][1:].replace(".","").isnumeric()):
                    line[j]=0.0-float(line[j][1:])
            else:
                if(line[j].isnumeric()):
                    line[j]=int(line[j])
                elif(line[j].replace(".","").isnumeric()):
                    line[j]=float(line[j])
    return line

# Loading data from above logs to excel file
with xlsxwriter.Workbook(filepath) as workbook:
    # OLDW_COUNT_TREND
    workbook.add_vba_project(r'C:\Users\\'+UID+'\Downloads\current_vs_previous.bin')
    worksheet = workbook.add_worksheet(name="OLDW_COUNT_TREND")
    stdin,stdout,stderr=ssh_client.exec_command("cat {}".format(file_names["OLDW_COUNT_TREND"][1]))
    out=stdout.readlines()
    for i in range(len(out)):
        cols=out[i][:-1].split("|")
        for j in range(len(cols)):
            cols[j]=cols[j].strip()
        worksheet.write_row(i,0,cols)
    
    # ADD_CLAIM , DERIVED_COLUMNS , DATA_SOURCE_CODE    
    stdin,stdout,stderr=ssh_client.exec_command("cat {}".format(file_names["ADD_DERIVED_DATA_SOURCE"][1]))
    out=stdout.readlines()
    sheets=["DERIVED_COLUMNS","ADD_CLAIM","DATA_SOURCE_CODE"]
    headers=[["DESCRIPTION","  TABLENAME    ","       COLUMNNAME  ","       DISTINCT_VALUES    ","  LIVE_TABLE_CNT  ","	  LIVE_DISTINCT_CNT   ","       NONLIVE_TABLE     ","          NONLIVE_COLUMN       ","      NONLIVE_DISTINCT_VALUES    ","  NONLIVE_TABLE_CNT  ","  NONLIVE_DISTINCT_CNT  ","  ROW_LEVEL_CHANGE  "," ROW_LEVEL_CHANGE(PERCENTAGE) ","    LIVE_PERCENT  ","   NONLIVE_PERCENT","DIFFERENCE"," COMMENTS "],["DESCRIPTION","TBL_NAME","DISTINCT_VALUE (D_CLAIM_AMT_IND)","  LIVE_CNT_WITH_FILTER ","   LIVE_CNT   "," LIVE_PERCENT ","  NONLIVE_CNT_WITH_FILTER  ","  NONLIVE_CNT  "," ROW_LEVEL_CHANGE ","ROW_LEVEL_CHANGE(PERCENTAGE)"," NONLIVE_PERCENT ","   DIFFERENCE   ","COMMENTS"],["   DESCRIPTION    ","        TBL_NAME         "," DATA_SOURCE_CODE ","  LIVE_CNT_WITH_FILTER  ","  LIVE_CNT   ","  LIVE_PERCENT   (filtered records) ","  NONLIVE_CNT_WITH_FILTER  ","  NONLIVE_CNT  "," ROW_LEVEL_CHANGE ","ROW_LEVEL_CHANGE(PERCENTAGE)"," NONLIVE_PERCENT (filtered records)","   DIFFERENCE   (NONLIVE-LIVE)","     COMMENTS    "]]
    count=0
    worksheet = workbook.add_worksheet(name=sheets[count])
    worksheet.write_row(0,0,headers[count])
    row=1
    for i in out:
        line=i.split("|")
        worksheet.write_row(row,0,line)
        if("rows" in i and count<2):
            count+=1
            worksheet = workbook.add_worksheet(name=sheets[count])
            worksheet.write_row(0,0,headers[count])
            row=1
        row+=1
        
    # MC,MCMPARTD
    stdin,stdout,stderr=ssh_client.exec_command("cat {}".format(file_names["MC_MCMPARTD"][1]))
    out=stdout.readlines()
    worksheet = workbook.add_worksheet(name="MC,MCM_PARTD_DISTRIBUTION")
    headers=["LIVE_TABLE","LIVE_COLUMN","LIVE_DISTINCT_VALUES","LIVE_DISTINCT_CNT","LIVE_TABLE_CNT","LIVE_PERCENT","NONLIVE_DISTINCT_CNT","NONLIVE_TABLE_CNT","ROW_LEVEL_CHANGE","ROW_LEVEL_CHANGE(PERCENTAGE)","NONLIVE_PERCENT","DIFFERENCE%","COMMENTS"]
    worksheet.write_row(0,0,headers)
    for i in range(len(out)):
        line=out[i].split("|")
        worksheet.write_row(i+1,0,line)
    
    # DISTRIBUTION_BASED_ON_DATES
    stdin,stdout,stderr=ssh_client.exec_command("cat {}".format(file_names["DISTR_BASED_ON_DATES"][1]))
    out=stdout.readlines()
    worksheet = workbook.add_worksheet(name="DISTRIBUTION_BASED_ON_DATES")
    i=0
    row=1
    headers=["   DESCRIPTION   ","    TBL_NAME    "," YEAR_MM "," LIVE_CNT_WITH_FILTER1 ","  LIVE_CNT  "," LIVE_PERCENT "," NONLIVE_CNT_WITH_FILTER "," NONLIVE_CNT "," NONLIVE_PERCENT ","   DIFF   "]
    worksheet.write_row(0,0,headers)
    while(i<len(out)):
        line=out[i].strip()
        if("TBL_NAME" not in line):
            line=line.split("|")
            if(len(line)>8):
                line=clean_num(line)
                worksheet.write_row(row,0,line)
                row+=1
        i+=1

#Getting VBA file from server to local

def getvba():
    remotefile=oldwpath+"current_vs_previous.bin"
    local_vba=  r"C:\Users\\"+UID+"\Downloads\current_vs_previous.bin"
    sftp = ssh_client.open_sftp()
    try:
        sftp.get(remotefile,local_vba)
        sftp.close()
        print("VBA file saved in local")
    except Exception as e:
        print("Getting VBA file to Local failed")
        print("Error :")
        print(e)

getvba()

#Running macros in excel file
def file_processing(filepath):
    print("Processing ",filepath)
    xl=win32com.client.Dispatch("Excel.Application")
    wb1=xl.Workbooks.Open(Filename=filepath)
    try:
        xl.Visible=True
        xl.Application.Run(filename+"!Module1.oldw_count_trend")
        xl.Application.Run(filename+"!Module2.derived_columns")
        xl.Application.Run(filename+"!Module3.data_source_code")
        xl.Application.Run(filename+"!Module4.MC_MCMPD")
        xl.Application.Run(filename+"!Module5.distr_based_on_dates")
        xl.Application.Run(filename+"!Module6.add_claim")
        
        wb1.Close(SaveChanges=1)
        print("Success! file : "+filepath)
        transfer(oldwpath+filename,filepath)
    except Exception as er:
        print("Macro processing failed")
        print(er)
        xl.Workbooks(1).Close(SaveChanges=0)
        xl.Application.Quit()
    xl.Quit

def transfer(oldw_final_file,final_file):
    sftp = ssh_client.open_sftp()
    sftp.put(final_file,oldw_final_file)
    sftp.close()
    print("File transferred successfully to OLABS server")
    send_mail(oldw_final_file)

# Sending report over mail
def send_mail(oldw_final_file):
    sub="Current_vs_Previous_Month "
    stdin,stdout,stderr=ssh_client.exec_command("sh /OLABS_QA/scripts/SOPmetricsmail.sh {} {}".format(sub,oldw_final_file))
    out=stderr.readlines()
    if(len(out)==0):
        print("Report sent over mail successfully")
    else:
        print("Report mail failed")
        print(out)

file_processing(filepath)